# Create catalogue of teleseismic events suitable for XKS splitting

SplitRacer provides utilities for creating a catalogue of suitable events via an interface to the FDSN request servers. In order to use SplitRacer with data from a locally stored data archive, one needs to create this event catalogue.

The general idea is to loop over each station and create an ObsPy catalogue of events between epicentral distances of 85 to 180 degrees. These catalogues are stored individually, so as to ensure waveform data is only cut for suitable event-station pairs.

In [8]:
# --- Imports ---
import pathlib

import numpy as np
import obspy
from obspy.clients.fdsn import Client
import pandas as pd

## Setup client and read in station information

Station information needs to be parsed into a pandas DataFrame. This can either be from a .csv file or using the provided utility function to parse the required information from a response inventory.

In [9]:
# --- Setup ---
# Initialise a client object
client = Client("IRIS")

# Specify the time period over which to query the online catalogue
starttime = obspy.UTCDateTime("2009-07-28")
endtime = obspy.UTCDateTime("2012-200")

In [10]:
# --- Station information ---
stations = pd.read_csv("/path/to/station_file")

In [ ]:
# --- Station inventory ---
inventory = obspy.read_inventory("/path/to/inventory_file")

## Setup parameters

Here we specify the parameters used to filter/select the events for the catalogue. Typically, core phases appear at epicentral distances beyond ~85 degrees, though each phase (e.g. SKS, SKKS, PKS, PKKS, SKIKS, PKIKS) is most suitable in different ranges of angle bands.

The best earthquakes for this type of analysis are those that are intermediate- or deep-focus and have magnitude in excess of ~ 5.8.

In [11]:
min_magnitude = 5.8
min_epicentral_distance = 85
max_epicentral_distance = 180

output_path = pathlib.Path.cwd() / "catalogues"
output_path.mkdir(parents=True, exist_ok=True)

## Generate catalogues for each station

Generate individual event catalogues for each station. If you wish to run the analysis under one directory, you should merge the resultant catalogues. Otherwise, keeping each catalogue distinct should minimise the amount of attempted pre-processing.

You may find the process can time out. A list of processed stations is constructed so the process can be restarted without having to re-download event catalogues that already exist.

If a catalogue of events for a station fails, I advise you to just take the event catalogue for the nearest station, or try to run the `get_events` command again for that specific station.

### From `pandas.DataFrame` of stations

In [12]:
catalogues, stations_processed = [], []

In [13]:
for _, station in stations.iterrows():
    if station.Name in stations_processed:
        continue
    print(f"Working on {station.Name}...")
    try:
        catalogue = client.get_events(
            starttime=starttime,
            endtime=endtime,
            minmagnitude=min_magnitude, 
            latitude=station.Latitude,
            longitude=station.Longitude,
            minradius=min_epicentral_distance,
            maxradius=max_epicentral_distance)
    except:
        print("    ...catalogue creation failed. Continuing.")
    print(f"    ...catalogue contains {len(catalogue)} events.")
    catalogues.append(catalogue)

    fname = (f"{starttime.year}_{starttime.julday:03d}-"
             f"{endtime.year}_{endtime.julday:03d}_"
             f"{min_epicentral_distance}-{max_epicentral_distance}_"
             f"{station.Name}_m{min_magnitude}_XKS_Catalogue.xml")
    catalogue.write(str(output_path / fname), format="QUAKEML")
    stations_processed.append(station.Name)


Working on K006...
    ...catalogue contains 574 events.
Working on K008...
    ...catalogue contains 573 events.
Working on K009...
    ...catalogue contains 572 events.
Working on K010...
    ...catalogue contains 574 events.
Working on K020...
    ...catalogue contains 574 events.
Working on K025...
    ...catalogue contains 574 events.
Working on K030...
    ...catalogue contains 573 events.
Working on K040...
    ...catalogue contains 573 events.
Working on K050...
    ...catalogue contains 573 events.
Working on K055...
    ...catalogue contains 574 events.
Working on K060...
    ...catalogue contains 574 events.
Working on K070...
    ...catalogue contains 573 events.
Working on K071...
    ...catalogue contains 573 events.
Working on K080...
    ...catalogue contains 573 events.
Working on K085...
    ...catalogue contains 574 events.
Working on K090...
    ...catalogue contains 574 events.
Working on K100...
    ...catalogue contains 573 events.
Working on K110...
    ...catal

### From `obspy.Inventory` object

In [ ]:
catalogues, stations_processed = [], []

In [ ]:
for network in inventory:
    for station in network:
        if station.Name in stations_processed:
            continue
        print(f"Working on {station.Name}...")
        catalogue = client.get_events(
            starttime=starttime,
            endtime=endtime,
            minmagnitude=min_magnitude, 
            latitude=station.Latitude,
            longitude=station.Longitude,
            minradius=min_epicentral_distance,
            maxradius=max_epicentral_distance)
        print(f"    ...catalogue contains {len(catalogue)} events.")
        catalogues.append(catalogue)

        catalogue.write(str(output_path / (f"{min_epicentral_distance}-"
                                           f"{max_epicentral_distance}_"
                                           f"{station.Name}_m{min_magnitude}"
                                           "_XKS_Catalogue.xml")),
                        format="QUAKEML")
        stations_processed.append(station.Name)

## Write the events to file

### Combine into single catalogue before writing

In [14]:
final_catalogue = obspy.Catalog()
event_ids = []
for catalogue in catalogues:
    for event in catalogue:
        if event.resource_id not in event_ids:
            print(f"Adding new event - {event.resource_id}")
            event_ids.append(event.resource_id)
            final_catalogue += event

Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=8761009
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=3343450
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=3343342
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=8755727
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=8751710
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=3342649
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=8742073
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=8736016
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=3341014
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=8723960
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=3340943
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=8720345
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=8720170

Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2841485
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2841349
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2841295
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2841294
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2841270
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2840991
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2840937
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2840930
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2840846
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2840820
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2840778
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2840676
Adding new event - smi:service.iris.edu/fdsnws/event/1/query?eventid=2840563

### Create event list file

In [16]:
# --- Functions ---
def write_eventlist(fname, catalogue):
    """
    Short utility function to export an ObsPy Catalog to the SplitRacer event
    list file format.

    Parameters
    ----------
    fname : str
        Name of event list file.
    catalogue : `obspy.Catalog` object
        A catalogue of `obspy.Event` objects, which contain event information.

    """

    with open(fname, "w") as f:
        for event in catalogue:
            origin = event.origins[0]
            magnitude = event.magnitudes[0]
            out = fmt.format(str(origin.time),
                             origin.latitude,
                             origin.longitude,
                             origin.depth/1000,
                             magnitude.mag,
                             magnitude.magnitude_type)
            print(out, file=f)
            
line = "time |  latitude |      longitude |     depth | mag |   magType"
fmt = "{} {:3.3f} {:3.3f} {} {:3.1f} {}"

In [17]:
# For the entire catalogue
write_eventlist("eventlist_all.txt", final_catalogue)   

In [18]:
# Or each station catalogue separately
for station, catalogue in zip(stations_processed, catalogues):
    write_eventlist(f"eventlist_{station}.txt", catalogue)  